### Importing Packages

In [20]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

load_dotenv()

True

### Model Setup

In [6]:
model = ChatGroq(
    model="meta-llama/llama-4-maverick-17b-128e-instruct",
    temperature=0.7,
    max_tokens=1024,
)

### StrOuput Parser

In [12]:
parser = StrOutputParser()

template1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

template2 = PromptTemplate(
    template='Write a 5 line summary on the following text. /n {text}',
    input_variables=['text']
)

chain = template1 | model | parser | template2 | model | parser

In [9]:
result = chain.invoke({'topic':'black hole'})
print(result)

Here is a 5-line summary of the text:

A black hole is a region in space with such strong gravity that nothing, including light, can escape. Black holes are formed when a massive star collapses, causing a supernova explosion, and can be characterized by their mass, spin, and charge. There are four types of black holes, including stellar, supermassive, intermediate-mass, and primordial black holes. Astronomers detect black holes by observing their effects on the surrounding environment, such as X-rays, gamma rays, and gravitational waves. The study of black holes has led to a deeper understanding of the fundamental laws of physics, including general relativity and quantum mechanics.


### JSONOuput Parser

In [13]:
parser = JsonOutputParser()

template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = template | model | parser

In [14]:
result = chain.invoke({'topic':'black hole'})
print(result)

{'facts': [{'fact': 'Black holes are regions of spacetime where gravity is so strong that nothing, not even light, can escape.', 'id': 1}, {'fact': 'The boundary surrounding a black hole is called the event horizon, beyond which anything that enters cannot escape.', 'id': 2}, {'fact': 'Black holes come in various sizes, ranging from small, stellar-mass black holes formed from star collapse, to supermassive black holes found at galaxy centers.', 'id': 3}, {'fact': "The existence of black holes was predicted by Albert Einstein's theory of general relativity, and the first modern black hole candidate was discovered in 1971.", 'id': 4}, {'fact': 'The information paradox surrounding black holes questions what happens to the information contained in matter that falls into a black hole, as it appears to be lost forever.', 'id': 5}]}


### StructuredOutput Parser

In [16]:
schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema)

template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

In [17]:
result = chain.invoke({'topic':'black hole'})
print(result)

{'fact_1': 'A black hole is a region in space where the gravitational pull is so strong that nothing, including light, can escape.', 'fact_2': 'Black holes are formed when a massive star collapses in on itself, causing a massive amount of matter to be compressed into an incredibly small space.', 'fact_3': 'The event horizon of a black hole marks the boundary beyond which anything that enters cannot escape, and it is directly related to the mass of the black hole.'}


### PydanticOutput Parser

In [34]:
class Person(BaseModel):
    name: str = Field(description='Name of the person')
    age: int = Field(gt=18, description='Age of the person')
    city: str = Field(description='Name of the city the person belongs to')

parser = PydanticOutputParser(pydantic_object=Person)

template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

In [35]:
result = chain.invoke({'place':'India'})
print(result)

name='Rohan Desai' age=32 city='Mumbai'
